In [3]:
#ROLLL NO: 22K-5095,  22K-5030


import heapq
import tkinter as tk
import random
import time
import networkx as nx
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt


GRID_SIZE = 8
GOAL_STATE = list(range(1, GRID_SIZE * GRID_SIZE)) + [0]


class Node:
    def __init__(self, state, parent=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.g = g     
        self.h = h      
        self.f = g + h 

    def __lt__(self, other):
        return self.f < other.f


class Environment:
    def __init__(self):
        self.grid_size = GRID_SIZE
        self.goal_state = GOAL_STATE

    def calculate_heuristic(self, state):
        
        distance = 0
        for i in range(self.grid_size * self.grid_size):
            tile = state[i]
            if tile == 0:
                continue
            goal_pos = self.goal_state.index(tile)
            current_row, current_col = divmod(i, self.grid_size)
            goal_row, goal_col = divmod(goal_pos, self.grid_size)
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance

    def find_legal_moves(self, state):
        
        empty_idx = state.index(0)
        row, col = divmod(empty_idx, self.grid_size)
        neighbors = []
        
       
        directions = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1)
        }
        
        for dr, dc in directions.values():
            new_row = row + dr
            new_col = col + dc
            if 0 <= new_row < self.grid_size and 0 <= new_col < self.grid_size:
                new_idx = new_row * self.grid_size + new_col
                new_state = list(state)
                new_state[empty_idx], new_state[new_idx] = new_state[new_idx], new_state[empty_idx]
                neighbors.append(new_state)
                
        return neighbors

    def check_goal_state(self, state):
        return state == self.goal_state

    def create_scrambled_puzzle(self, steps=50):
        
        state = list(self.goal_state)
        for _ in range(steps):
            neighbors = self.find_legal_moves(state)
            state = random.choice(neighbors)
        return state


class AIAgent:
    def __init__(self, env):
        self.env = env
        self.search_graph = nx.DiGraph() 


    def perform_a_star(self, start_state):
        open_heap = []
        start_node = Node(start_state, None, 0, self.env.calculate_heuristic(start_state))
        heapq.heappush(open_heap, start_node)
        
        closed_set = set()
        visited_states = []
        

        while open_heap:
            
            current_node = heapq.heappop(open_heap)
            visited_states.append(current_node.state)
            self.search_graph.add_node(tuple(current_node.state))
            if self.env.check_goal_state(current_node.state):
                return self.construct_solution_path(current_node), visited_states
            
            closed_set.add(tuple(current_node.state))
            
            for neighbor_state in self.env.find_legal_moves(current_node.state):
                if tuple(neighbor_state) in closed_set:
                    continue
                
                g = current_node.g + 1
                h = self.env.calculate_heuristic(neighbor_state)
                neighbor_node = Node(neighbor_state, current_node, g, h)
                
                in_open = False
                for node in open_heap:
                    if node.state == neighbor_state and node.g <= g:
                        in_open = True
                        break
                if not in_open:
                    heapq.heappush(open_heap, neighbor_node)
                    self.search_graph.add_edge(tuple(current_node.state), tuple(neighbor_state))
        
        return None, visited_states  

    def construct_solution_path(self, node):
        path = []
        while node:
            path.append(node.state)
            node = node.parent
        return path[::-1]


class PuzzleGUI:
    def __init__(self, root):
        self.root = root
        self.env = Environment()
        self.agent = AIAgent(self.env)
        self.current_state = None
        
        self.initialize_controls()
        self.initialize_grid()


    def initialize_controls(self):


        control_frame = tk.Frame(self.root)
        control_frame.pack(pady=10)
        self.steps_label = tk.Label(control_frame, text="Scramble Steps:", font=("Arial", 16))
        self.steps_label.pack(side=tk.LEFT)
        self.steps_entry = tk.Entry(control_frame, width=5, font=("Arial", 16))
        self.steps_entry.insert(0, "50")
        self.steps_entry.pack(side=tk.LEFT, padx=5)
        
        self.scramble_btn = tk.Button(control_frame, text="Scramble", command=self.scramble_puzzle,
                                      font=("Arial", 16), bg="blue", fg="white")
        self.scramble_btn.pack(side=tk.LEFT, padx=5)
        self.solve_btn = tk.Button(control_frame, text="Solve", command=self.solve_puzzle,
                                   font=("Arial", 16), bg="green", fg="white", state=tk.DISABLED)
        self.solve_btn.pack(side=tk.LEFT, padx=5)


    def initialize_grid(self):
        
        self.grid_frame = tk.Frame(self.root)
        self.grid_frame.pack()
        
        self.tiles = []
        for i in range(GRID_SIZE * GRID_SIZE):
            row, col = divmod(i, GRID_SIZE)
            tile = tk.Label(self.grid_frame, text="", font=("Arial", 14), 
                            width=4, height=2, relief="raised", bg="white", borderwidth=2)
            tile.grid(row=row, column=col, padx=1, pady=1)
            self.tiles.append(tile)



    def render_grid(self, state, highlight_index=None):
        for i in range(GRID_SIZE * GRID_SIZE):
            text = str(state[i]) if state[i] != 0 else ""
       
            bg = "lightblue" if highlight_index is not None and i == highlight_index else "white"
            self.tiles[i].config(text=text, bg=bg)
        self.root.update()
        time.sleep(0.09)



    def scramble_puzzle(self):
        try:
            steps = int(self.steps_entry.get())
        except ValueError:
            steps = 50  
        self.current_state = self.env.create_scrambled_puzzle(steps)
        self.render_grid(self.current_state)
        self.solve_btn.config(state=tk.NORMAL)


    def solve_puzzle(self):
        solution_path, _ = self.agent.perform_a_star(self.current_state)
        
        if solution_path:
           

            for state in solution_path:
                empty_idx = state.index(0)
                self.render_grid(state, highlight_index=empty_idx)
                time.sleep(0.3)
            self.display_search_graph()
            tk.Label(self.root, text="Solution Found!", font=("Arial", 16), fg="green").pack(pady=5)
        else:
            tk.Label(self.root, text="No Solution Found!", font=("Arial", 16), fg="red").pack(pady=5)


    def display_search_graph(self):
        try:
            plt.figure(figsize=(12, 8))
            pos = nx.spring_layout(self.agent.search_graph)
            nx.draw(self.agent.search_graph, pos, with_labels=False, node_size=20, 
                    arrowsize=5, width=0.5, node_color="skyblue")
            plt.title("State Space Exploration Graph")
            plt.show()
        except Exception as e:
            print("Error displaying graph:", e)
        



if __name__ == "__main__":
    root = tk.Tk()
    root.title("8x8 (64 Puzzle) Solver - Graph Traversal Visualization")
    app = PuzzleGUI(root)
    root.mainloop()

KeyboardInterrupt: 